In [1]:
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import matplotlib.animation as animation
from tqdm import tqdm

## Ejercicio 4

In [3]:
A=np.array([[1,0,0],
           [5,1,0],
           [-2,3,-1]])
B=np.array([[4,-2,1],
           [0,3,7],
           [0,0,2]])

In [4]:
def producto(A, B):
    tam_A = np.shape(A)
    tam_B = np.shape(B) 

    if tam_A[1] != tam_B[0]:
        raise Exception("Las matrices no se pueden multiplicar.")
        return False
    else:
        C = np.empty((tam_A[0], tam_B[1]))
        for filas in range(0,tam_A[0]):
            for col in range(0,tam_B[1]):
                C[filas,col] = np.dot(A[filas,:], B[:,col])
        return C

print(producto(A,B))

[[ 4. -2.  1.]
 [20. -7. 12.]
 [-8. 13. 17.]]


## Ejercicio 7

In [ ]:
M = np.array([[3,-1,-1],[-1,3,1],[2,1,4]],dtype=np.float32)
b = np.array([1,3,7],dtype=np.float32)
print(M)


In [58]:
def Jacobi(M,b,itmax):
    x_k = np.zeros_like(b)

    for it in range(itmax):
        x_k1 = np.zeros_like(b)
        for i in range(x_k1.shape[0]):
            suma = 0
            for j in range(M.shape[0]):
                if i != j:
                    suma += M[i][j]*x_k[j]
            
            x_k1[i] = (1./M[i][i])*(b[i] - suma)
        x_k = x_k1
    return x_k

In [59]:
Jacobi(M,b,50)

array([1., 1., 1.], dtype=float32)

In [100]:
def SOR(M,b,itmax,w,tolerancia):
    x_old = np.zeros_like(b)
    x_new = np.zeros_like(b)*np.nan

    for it in range(itmax):

        for i in range(b.shape[0]):
            suma_new = 0
            suma_old = 0

            for j in range(i):
                suma_new += M[i,j]*x_new[j]

            for j in range(i+1,b.shape[0]):
                suma_old += M[i,j]*x_old[j]
                    
            x_new[i] = (1-w)*x_old[i] + (w/M[i,i])*(b[i] - suma_new - suma_old)

        
        if np.linalg.norm(M@x_new-b) < tolerancia:
            break

        x_old = x_new
    return x_new,it

In [96]:
np.linalg.solve(M,b)

array([1., 1., 1.], dtype=float32)

In [154]:
SOR(M,b,100,1.2,1e-18)

(array([1., 1., 1.], dtype=float32), 33)

In [ ]:
SOR(M,b,1000,1.5,1e-3)

In [150]:
dominio = np.linspace(1,2,10,endpoint=False)

In [151]:
argmin = 0
num_it = 1000

for d in dominio:

    _,it = SOR(M,b,1000,d,1e-3)

    if it < num_it:
        num_it = it
        argmin = d


C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:20: RuntimeWarning: overflow encountered in matmul
  if np.linalg.norm(M@x_new-b) < tolerancia:
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:12: RuntimeWarning: overflow encountered in scalar multiply
  suma_new += M[i,j]*x_new[j]
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:20: RuntimeWarning: invalid value encountered in matmul
  if np.linalg.norm(M@x_new-b) < tolerancia:
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:17: RuntimeWarning: invalid value encountered in scalar subtract
  x_new[i] = (1-w)*x_old[i] + (w/M[i,i])*(b[i] - suma_new - suma_old)


In [152]:
import pandas as pd
arr = []
for w in np.linspace(1.1,2,100,endpoint=False)[:-1]:
    _,it = SOR(M,b,1000,w,1e-3)
    arr.append([w,it])

res = pd.DataFrame(arr)
res.columns = ['omega','num_it']
res.sort_values('num_it').head()

C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:20: RuntimeWarning: overflow encountered in matmul
  if np.linalg.norm(M@x_new-b) < tolerancia:
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:12: RuntimeWarning: overflow encountered in scalar multiply
  suma_new += M[i,j]*x_new[j]
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:20: RuntimeWarning: invalid value encountered in matmul
  if np.linalg.norm(M@x_new-b) < tolerancia:
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:17: RuntimeWarning: invalid value encountered in scalar subtract
  x_new[i] = (1-w)*x_old[i] + (w/M[i,i])*(b[i] - suma_new - suma_old)
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:17: RuntimeWarning: overflow encountered in cast
  x_new[i] = (1-w)*x_old[i] + (w/M[i,i])*(b[i] - suma_new - suma_old)
C:\Users\Flutt\AppData\Local\Temp\ipykernel_5952\1769158062.py:15: RuntimeWarning: invalid value encountered in scalar add
  suma_old += M[i,j]*

,omega,num_it
0,1.100,10
1,1.109,10
2,1.118,11
3,1.127,11
4,1.136,12


# Ejercicio 12

In [101]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [103]:
G = np.array([lambda x, y: np.log(x**2 + y**2) - np.sin(x*y) - np.log(2) - np.log(np.pi),
              lambda x, y: np.exp(x-y) + np.cos(x*y)])
F = np.array([
    lambda x,y: G[0](x,y)**2,
    lambda x,y: G[1](x,y)**2,
])

In [104]:
def dominio_funcion1(x,y):
    if x**2 + y**2 <= 0:
            return 0 
    else:
        return np.log(x**2 + y**2) - np.sin(x * y)

In [106]:
def derivada_x(f,x,y,h):
    return (f[0](x+h,y) - f[0](x-h,y))/(2*h)

def derivada_y(f,x,y,h):
    return (f[1](x,y+h) - f[1](x,y-h))/(2*h)

def Gradient(f,x,y):
    dx = derivada_x(f,x,y,1e-5)
    dy = derivada_y(f,x,y,1e-5)
    return np.array([dx, dy])

x0, y0 = 2, 2
Gradient(F, x0, y0)





array([3.60867178, 0.35578073])

In [120]:
#Gradient descent 2D
def Minimizer(f,x0, N = 500, gamma = 0.01):
    
    r = np.zeros((N,2))
    r[0] = x0
    
    Grad = np.zeros((N,2))
    Grad[0] = Gradient(f,r[0,0],r[0,1])
    
    # We save the gradient in each step

    for i in tqdm(range(1,N)):
        r[i] = r[i-1] - gamma*Gradient(f,r[i-1,0],r[i-1,1])
        Grad[i] = Gradient(f,r[i-1,0],r[i-1,1])
        
        
    return r,Grad

In [121]:
xmin = Minimizer(F,[2,2])[0][-1]

100%|██████████| 499/499 [00:00<00:00, 8995.82it/s]


In [122]:
GetF(G,xmin)

array([-1.62092562e-14, -2.07113438e-10])

In [111]:
def GetF(G,r):
    
    n = r.shape[0]
    
    v = np.zeros_like(r)
    
    for i in range(n):
        v[i] = G[i](r[0],r[1])
       
    return v


In [126]:
def GetJacobian(f,r,h=1e-6):
    
    n = r.shape[0]
    
    J = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            
            rf = r.copy()
            rb = r.copy()
            
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            
            J[i,j] = ( f[i](rf[0],rf[1]) - f[i](rb[0],rb[1])  )/(2*h)
            
            
    
    return J

In [127]:
J = Minimizer(G,np.zeros(2),100,0.01)

100%|██████████| 99/99 [00:00<00:00, 15716.73it/s]


In [145]:
def NewtonRaphson(G,r,itmax=1000,error=1e-9):
    
    it = 0
    d = 1.
    dvector = []
    
    while d > error and it < itmax:
        
        # Vector actual
        rc = r
        
        F = GetF(G,rc)
        J = GetJacobian(G,rc)
        #J = Minimizer(G,2,0.01)
                  
        r = rc - np.dot(np.linalg.inv(J),F)
        
        diff = r - rc
        
        d = np.max( np.abs(diff) )
        
        dvector.append(d)
        #print(dvector)
        
        it += 1
    
    
    return r,dvector

In [146]:
r,dvector = NewtonRaphson(G,np.array([2,2.]))

In [147]:
r

array([1.77245385, 1.77245385])

In [148]:
GetF(F,r)

array([4.93038066e-32, 0.00000000e+00])